# Train a deep CNN on XPS data on local machine

In [ ]:
In this notebook, we will train a deep convolutional network on iron XPS spectra using the local CPU/GPU.

## Setup

In [ ]:
# Imports
%matplotlib inline
import os
import datetime
import pytz
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Run tensorflow on local CPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# Import custom classes
try:
    import  importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print('\n Classes were reloaded.')
except:
    import xpsdeeplearning.network.classifier as classifier
    import xpsdeeplearning.network.utils as clfutils
    print('\n Classes were loaded.')

## Initial run

### Setting up the parameters & folder structure

In [ ]:
np.random.seed(502)
time =  datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
model_type = 'CNN_simple'
model_name = 'Fe_single_4_classes_CNN_simple'

label_values = ['Fe metal', 'FeO', 'Fe3O4', 'Fe2O3']

clf = classifier.Classifier(time = time,
                           model_type = model_type,
                           model_name = model_name,
                           labels = label_values)

### Load and inspect the data

In [ ]:
input_filepath = r'C:\Users\pielsticker\Simulations\20200605_iron_single_small.h5'   
train_test_split = 0.2
train_val_split = 0.2
no_of_examples = 1000

X_train, X_val, X_test, y_train, y_val, y_test = clf.load_data_preprocess(input_filepath = input_filepath,
                                                                          no_of_examples = no_of_examples,
                                                                          train_test_split = train_test_split,
                                                                          train_val_split = train_val_split)
# Check how the examples are distributed across the classes:
class_distribution = clf.check_class_distribution()
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 9, dataset = 'train')  

### Build the model

In [ ]:
learning_rate = 1e-05

clf.build_model(learning_rate)
clf.summary()
clf.save_and_print_model_image()

### Training

In [ ]:
epochs = 2
batch_size = 32

hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs, 
                 batch_size = batch_size,
                 verbose = 1)

### Graphs

In [ ]:
graph = clfutils.TrainingGraphs(clf.history,
                                clf.model_name,
                                clf.time) 

### Evaluation on test data

In [ ]:
score = clf.evaluate()
test_loss, test_accuracy = score[0], score[1]
print('Test loss: ' + str(np.round(test_loss, decimals=3)))
print('Test accuracy: ' + str(np.round(test_accuracy, decimals=3)))

### Prediction on test data

In [ ]:
pred_train, pred_test = clf.predict()
pred_train_classes, pred_test_classes = clf.predict_classes()

### Show some predictions

#### Training data

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'train', with_prediction = True)  

#### Test data

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

### Saving data

In [ ]:
clf.save_model()
clf.shelve_results(full = False)  

### Report

In [ ]:
dir_name = clf.time + '_' + clf.model_name
rep = clfutils.Report(dir_name)  
rep.write()   

## Continue training 

In [ ]:
# Reload and train for more epochs
clf.load_model(from_path = False)
# model_path = r'C:\Users\pielsticker\Lukas\MPI-CEC\Projects\xpsdeeplearning\saved_models\20200608_17h51m_Fe_single_4_classes_CNN_simple' 
# clf.load_model(from_path = True, model_path = model_path)
new_learning_rate = 1e-04
epochs = 2
batch_size = 32
hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs,
                 batch_size = batch_size, 
                 new_learning_rate = new_learning_rate) # Learning rate can be changed for retraining

from tensorflow.keras import backend as K
print('New learning rate: ' +\
      str(K.eval(clf.model.optimizer.lr)))

graphs = clfutils.TrainingGraphs(hist,clf.model_name, clf.time)

score = clf.evaluate()
test_loss, test_accuracy = score[0], score[1]

pred_train, pred_test = clf.predict()
pred_train_classes, pred_test_classes = clf.predict_classes()
clf.plot_random(no_of_spectra = 6, dataset = 'train', with_prediction = True)  
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

clf.save_model()
clf.shelve_results(full = False)  

dir_name = clf.time + '_' + clf.model_name
rep = clfutils.Report(dir_name)  
rep.write()  


### Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"), include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time

save_notebook()
time.sleep(3)
current_file = 'train_local.ipynb'
output_file = os.path.join(clf.log_dir,'train_local_out.html')
output_HTML(current_file, output_file)